**1.转换文件格式**

In [1]:
import pandas as pd  
import re
  
# 读取Excel文件  
excel_file = 'D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/Teacher2.xlsx'  
df = pd.read_excel(excel_file)  
  
# 导出为CSV格式  
csv_file = 'D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/Teacher2.csv'  
df.to_csv(csv_file, index=False)
df

,序号,教师姓名,学位,本科毕业院校,研究生毕业院校,博士毕业院校,深造其他院校,研究方向,是否为博士后
0,1,郭春生,博士,北京师范大学,河北师范学院,中国社会科学院,NaN,科学社会主义与国际共产主义运动、中共党史,NaN
1,2,蒋 锐,NaN,NaN,NaN,NaN,NaN,当代国外社会主义 中国特色社会主义与传统文化 统一战线理论与政策 德国政治与文化,NaN
2,3,李 宏,NaN,NaN,NaN,NaN,NaN,欧洲政治、 欧洲政党,NaN
3,4,李亚洲,NaN,NaN,NaN,NaN,NaN,科学社会主义与国际共产主义运动、俄罗斯研究,NaN
4,5,刘玉安,NaN,NaN,NaN,NaN,NaN,国际政治理论 比较社会政策,NaN
...,...,...,...,...,...,...,...,...,...
98,99,金澄,NaN,NaN,NaN,NaN,NaN,区域经济学、发展经济学、公司治理、农业经济和经济史,NaN
99,100,贾小云,NaN,NaN,NaN,NaN,NaN,新技术的应用对心理、行为、思维和教育的影响，数字化智能化创新与发展，网络与新媒体研究,NaN
100,101,李爽,NaN,黑龙江大学,西南大学,奈梅亨大学,NaN,行为公共管理、政治心理学、社会心理学,NaN
101,102,王乙杰,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**2.提取所有院校**

In [2]:
# 提取相关列
School_index = df[['本科毕业院校', '研究生毕业院校', '博士毕业院校', '深造其他院校']]

# 使用stack方法将数据转换为一列
all_shcools = School_index.stack().reset_index(drop=True)

# 创建新的DataFrame
new_data = pd.DataFrame(all_shcools, columns=['院校'])

# 保存为新的CSV文件
new_data.to_csv('D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/all_school.csv', index=False)
new_data

,院校
0,北京师范大学
1,河北师范学院
2,中国社会科学院
3,山东大学
4,中国人民大学
...,...
89,黑龙江大学
90,西南大学
91,奈梅亨大学
92,普林斯顿大学


**3.根据院校查找每个院校的经纬度信息**

In [3]:
import csv
import re

data = """
北京师范大学: [116.3320, 39.9544],
河北师范学院: [115.5122, 38.9780],
中国社会科学院: [116.3883, 39.9282],
山东大学: [117.0190, 36.6685],
中国人民大学: [116.3473, 39.9610],
中国人民大学: [116.3473, 39.9610],
澳大利亚国立大学: [149.1150, -35.3035],
华南师范大学: [113.3676, 23.1622],
复旦大学: [121.5070, 31.3020],
青岛大学师范学院: [120.3821, 36.0865],
南京理工大学: [118.7963, 32.0250],
南京大学: [118.7960, 32.0603],
香港中文大学: [114.2170, 22.3959],
上海交通大学: [121.4825, 31.2001],
重庆大学: [106.5705, 29.5630],
耶鲁大学: [-72.9223, 41.3163],
哥德堡大学: [11.9746, 57.7089],
香港大学: [114.1628, 22.2797],
太原理工大学: [112.5495, 37.8628],
苏州大学: [120.5853, 31.2989],
多伦多大学: [-79.3957, 43.6629],
杜伦大学: [-1.5755, 54.7764],
南开大学: [117.1800, 39.1408],
韩国岭南大学国际教育院: [128.1025, 35.1444],
柏林自由大学: [13.2906, 52.4574],
山东师范大学: [117.1571, 36.6829],
北京外国语大学: [116.3489, 39.9658],
美国密苏里大学: [-92.3341, 38.9519],
北京大学: [116.4074, 39.9042],
德国弗莱堡大学: [7.8421, 47.9936],
复旦大学: [121.5070, 31.3020],
阿姆斯特丹自由大学: [4.8811, 52.3584],
兰州大学: [103.6177, 36.0569],
清华大学: [116.3290, 40.0056],
内蒙古科技大学: [111.6385, 40.8165],
特拉华大学: [-75.7490, 39.6838],
四川大学: [104.0666, 30.6502],
吉林大学: [125.3245, 43.8868],
哈尔滨工业大学: [126.6424, 45.7559],
中山大学: [113.3590, 23.1327],
伦敦大学国王学院: [-0.0904, 51.5108],
浙江大学: [120.1517, 30.2728],
中国矿业大学: [117.2050, 34.1870],
南开大学: [117.1800, 39.1408],
新加坡国立大学: [103.7764, 1.2966],
高丽大学: [126.9960, 37.5821],
华中师范大学: [114.3632, 30.5134],
北京大学: [116.4074, 39.9042],
伯明翰大学: [-1.5486, 52.3963],
哈尔滨工业大学: [126.6424, 45.7559],
华中农业大学: [114.3640, 30.5158],
昆士兰大学: [153.0187, -27.4979],
美国斯坦福大学: [-122.1697, 37.4275],
新加坡国立大学李光耀公共政策学院: [103.7764, 1.2966],
大连理工大学: [121.6079, 38.9040],
罗格斯大学: [-74.4451, 40.5015],
武汉大学: [114.4005, 30.5455],
西安交通大学: [108.9560, 34.2009],
黑龙江大学: [126.6329, 45.7475],
西南大学: [106.4930, 29.9733],
奈梅亨大学: [5.8450, 51.8424],
普林斯顿大学: [-74.6553, 40.3430],
中国人民大学: [116.3473, 39.9610],
"""

# 清洗和解析数据
rows = []
for line in data.strip().splitlines():
    # 使用正则表达式提取大学名称和经纬度
    match = re.match(r'(.+?): \[(.+), (.+)\]', line.strip())
    if match:
        name = match.group(1)
        longitude = match.group(2)
        latitude = match.group(3)
        rows.append([name, longitude, latitude])

# 写入 CSV 文件
with open('D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/school_location.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入列名
    writer.writerow(['Name', 'Longitude', 'Latitude'])
    # 写入数据行
    writer.writerows(rows)


**4.整合数据生成json文件**

In [4]:
import pandas as pd
import json

# 读取CSV文件
teacher_data = pd.read_csv("D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/Teacher2.csv")  # 包含教师信息
location_data = pd.read_csv("D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/school_location.csv")  # 包含院校位置


location_data.columns = location_data.columns.str.strip()  
teacher_data.columns = teacher_data.columns.str.strip()


teacher_data.rename(columns={
    "教师姓名": "name",
    "本科毕业院校": "undergraduate",
    "研究生毕业院校": "graduate",
    "博士毕业院校": "doctorate",
    "深造其他院校": "other institutions"
}, inplace=True)


# 数据整合
location_data = location_data.drop_duplicates(subset=['Name'])
locations_dict = location_data.set_index('Name')[['Longitude', 'Latitude']].to_dict(orient='index')

def get_institution_info(institution_name):
    if pd.isna(institution_name):
        return None
    info = locations_dict.get(institution_name, None)
    if info:
        return {"name": institution_name, "longitude": info['Longitude'], "latitude": info['Latitude']}
    return None

def process_teacher(row):
    return {
        "name": row['name'],
        "undergraduate": get_institution_info(row['undergraduate']),
        "graduate": get_institution_info(row['graduate']),
        "doctorate": get_institution_info(row['doctorate']),
        "other institutions": None if pd.isna(row['other institutions']) else row['other institutions']
    }

# 生成JSON数据
teachers_list = teacher_data.apply(process_teacher, axis=1).tolist()
final_json = {"teachers": teachers_list}


# 保存为JSON文件
with open("D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/teachers.json", "w", encoding="utf-8") as f:
    json.dump(final_json, f, ensure_ascii=False, indent=2)

print("JSON文件生成成功：D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/teachers.json")


JSON文件生成成功：D:/大三上/大数据分析与实践/实验/大作业/培养经历/数据集/teachers.json
